# Temporal Graph Analysis

## Prerequisites

First, we need to set up our Python environment that has PyTorch, PyTorch Geometric and PathpyG installed. Depending on where you are executing this notebook, this might already be (partially) done. E.g. Google Colab has PyTorch installed by default so we only need to install the remaining dependencies. The DevContainer that is part of our GitHub Repository on the other hand already has all of the necessary dependencies installed. 

In the following, we install the packages for usage in Google Colab using Jupyter magic commands. For other environments comment in or out the commands as necessary. For more details on how to install `pathpyG` especially if you want to install it with GPU-support, we refer to our [documentation](https://www.pathpy.net/dev/getting_started/). Note that `%%capture` discards the full output of the cell to not clutter this tutorial with unnecessary installation details. If you want to print the output, you can comment `%%capture` out.

In [ ]:
%%capture
# !pip install torch
# !pip install torch_geometric
# !pip install git+https://github.com/pathpy/pathpyG.git

## Motivation and Learning Objectives

In this tutorial we will introduce the representation of temporal graph data using the `TemporalGraph` class and how such data can be used to calculate shortest time respecting paths between nodes as well temporal node cemtralities.

In [ ]:
import os
import tempfile

import torch
from torch_geometric.data import Data

import pathpyG as pp

We can create a temporal graph object from a list of time-stamped edges. Since `TemporalGraph` is a subclass of the `Graph` class, the internal structures are very similar:

In [ ]:
tedges = [('a', 'b', 1),('a', 'b', 2), ('b', 'a', 3), ('b', 'c', 3), ('d', 'c', 4), ('a', 'b', 4), ('c', 'b', 4),
          ('c', 'd', 5), ('b', 'a', 5), ('c', 'b', 6)]
t = pp.TemporalGraph.from_edge_list(tedges)
print(t.mapping)
print(t.n)
print(t.m)

By default, all temporal graphs are directed. We can create an undirected version a temporal graph as follows:

In [ ]:
x = t.to_undirected()
print(x.mapping)
print(x.n)
print(x.m)

We can also directly create a temporal graph from an instance of `pyG.TemporalData`

In [ ]:
td = Data(edge_index=torch.Tensor([[0, 1, 2, 0], [1, 2, 3, 1]]).long(), time=torch.Tensor([0, 1, 2, 3]), num_nodes=4)
print(td)
t2 = pp.TemporalGraph(td)
print(t2)

We can restrict a temporal graph to a time window, which returns a temporal graph that only contains time-stamped edges in the given time interval.

In [ ]:
t1 = t.get_window(0, 4)
print(t1)
print(t1.m)
print(t1.start_time)
print(t1.end_time)

We can also extract a TemporalGraph object for a batch of temporal edges, which is defined by the start and end index of the edges defining the batch.

In [ ]:
t1 = t.get_batch(1, 6)
print(t1)
print(t1.m)
print(t1.start_time)
print(t1.end_time)

We can easily convert a temporal graph into a weighted time-aggregated static graph, where edge weights count the number of occurrences of an edge across all timestamps.

In [ ]:
g = t.to_static_graph(weighted=True)
print(g)

We can also aggregate a temporal graph within a certain time window:

In [ ]:
g = t.to_static_graph(time_window=(1, 3), weighted=True)
print(g)

Finally, we can use the class `RollingTimeWindow` to perform a rolling window analysis. The class returns an iterable object, where each iteration yields a time-aggregated weighted graph object as well as the corresponding time window.

In [ ]:
r = pp.algorithms.RollingTimeWindow(t, window_size=3, step_size=1, return_window=True)
for g, w in r:
    print("Time window ", w)
    print(g)
    print(g.data.edge_index)
    print("---")

We can visualize temporal graphs using the plot function just like static graphs:

In [ ]:
pp.plot(t);

The source nodes, destination nodes and timestamps of time-stamped edges are stored as a `pyG TemporalData` object, which we can access in the following way.

In [ ]:
t.data

In [ ]:
print(t.data.edge_index)

In [ ]:
print(t.data.time)

With the generator functions `edges` and `temporal_edges` we can iterate through the time-ordered (temporal) multi-edges of a temporal graph.

In [ ]:
for v, w in t.edges:
    print(v, w)

In [ ]:
for v, w, time in t.temporal_edges:
    print(v, w, time)

## Extracting Time-Respecting Paths in Temporal Networks

We are often interested in time-respecting paths in a temporal graph. A time-respecting path consists of a sequence of nodes $v_0,...,v_l$ where consecutive nodes are connected by time-stamped edges that occur (i) in the right temporal ordering, and (ii) within a maximum time difference of $\delta\in \N$. 

To calculate time-respecting paths in a temporal graph, we can construct a directed acyclic graph (DAG), where each time-stamped edge $(u,v;t)$ in the temporal graph is represented by a node and two nodes representing time-stamped edges $(u,v;t_1)$ and $(v,w;t_2)$ are connected by an edge iff $0 < t_2-t_1 \leq \delta$. This implies that (i) each edge in the resulting DAG represents a time-respecting path of length two, and (ii) time-respecting paths of any lenghts are represented by paths in this DAG.

We can construct such a DAG using the function `pp.algorithms.lift_order_temporal`, which returns an edge_index. We can pass this to the constructor of a `Graph` object, which we can use to visualize the resulting DAG.

In [ ]:
e_i = pp.algorithms.lift_order_temporal(t, delta=1)
dag_mapping = pp.IndexMap([f"{v}->{w}: {time}" for v, w, time in t.temporal_edges ])
dag = pp.Graph.from_edge_index(e_i, mapping=dag_mapping)
pp.plot(dag);

For $\delta=1$, this DAG with three connected components tells us that the underlying temporal graph has  the following time-respecting paths (of different lengths):

Length one:  
    a -> b  
    b -> a  
    b -> c  
    c -> b  
    c -> d  
    d -> c  

Length two:  
    a -> b -> a (twice, starting at time 2 and time 4)  
    b -> a -> b  
    a -> b -> c     
    b -> c -> b  
    c -> b -> a  
    d -> c -> d  

Length three:   
    a -> b -> a -> b  
    b -> a -> b -> a  
    a -> b -> c -> b  
    b -> c -> b -> a  
    
Length four:   
    a -> b -> a -> b -> a  
    a -> b -> c -> b -> a  

We can can use the function `pp.algorithms.temporal.temporal_shortest_paths` to calculate shortest time-respecting path distances between any pair of nodes. This also returns a predecessor matrix, which can be used to reconstruct all shortest time-respecting paths (in analogy to the Dijkstra algorithm for static graphs):

In [ ]:
dist, pred = pp.algorithms.temporal_shortest_paths(t, delta=1)
print(t.mapping)
print(dist)
print(pred)

In the example above, the four `inf` values indicate that there is no time-respecting paths between the four node pairs (a, d), (b, d), (d,a) and (d, b). This is not something we would expect based on the (strongly connected) topology of the time-aggregated graph, which is shown below:

In [ ]:
g = t.to_static_graph(weighted=True)
pp.plot(g);

## Reading and writing temporal graph data

Similar to simple graphs, we can read and write temporal graph data into various common formats. For example, we can easily convert our temporal graph to a Pandas DataFrame using the `temporal_graph_to_df` function and create a temporal graph from a DataFrame using the `df_to_temporal_graph` function:

In [ ]:
tedges = [('a', 'b', 1),('a', 'b', 2), ('b', 'a', 3), ('b', 'c', 3), ('d', 'c', 4), ('a', 'b', 4), ('c', 'b', 4),
              ('c', 'd', 5), ('b', 'a', 5), ('c', 'b', 6)]
t = pp.TemporalGraph.from_edge_list(tedges)
df = pp.io.temporal_graph_to_df(t)
print(df)

In [ ]:
t = pp.io.df_to_temporal_graph(df)
print(t)

In [ ]:
with tempfile.TemporaryDirectory() as tmpdirname:
    print('created temporary directory', tmpdirname)
    tmp_file = os.path.join(tmpdirname, 'test_graph.csv')
    pp.io.write_csv(t, path_or_buf=tmp_file)

    csv_t = pp.io.read_csv_temporal_graph(filename=tmp_file)
pp.plot(csv_t);

## Temporal Centralities in Empirical Temporal Networks

`pathpyG`'s ability to calculate (shortest) time-respecting paths enables us to calulate different notions of temporal centralities for nodes in empirial temporal networks. We can read an empirical temporal graph based on CSV data, where each line contains the source, target, and timestamp of an edge as comma-separated value:

In [ ]:
t_baboons = pp.io.read_netzschleuder_graph('sp_baboons', 'observational', time_attr='time')
t_baboons.data.time = (t_baboons.data.time - t_baboons.data.time.min()) // 60 # convert to minutes
print(t_baboons)

To calculate the temporal closeness centrality, which is defined based on the length of shortest time-respecting paths of a node to all other nodes, we can write the following: 

In [55]:
cl = pp.algorithms.centrality.temporal_closeness_centrality(t_baboons, delta=24*60)
print(cl)
node_size = {v: 15 * (x / max(cl.values())) for v, x in cl.items()}
pp.plot(t_baboons, node_size=node_size, node_color=t_baboons.nodes);

100%|██████████| 1308/1308 [00:00<00:00, 6649.44it/s]


{0: 356.4, 1: 390.5, 2: 385.0, 3: 392.33333333333337, 4: 378.4, 5: 399.66666666666663, 6: 374.0, 7: 352.0, 8: 357.5, 9: 352.0, 10: 297.0, 11: 0.0, 12: 374.0, 13: 366.66666666666663, 14: 352.0, 15: 407.0, 16: 363.0, 17: 352.0, 18: 385.0, 19: 348.33333333333337, 20: 174.16666666666666, 21: 145.5666666666667, 22: 159.50000000000003}


The definition of time-respecting paths depends on our maximum time difference parameter $\delta$, which implies that different values of this parameter also yield different centralities. This means that we can calculate temporal node centralities for different "time scales" of a temporal graph.

In [ ]:
cl = pp.algorithms.centrality.temporal_closeness_centrality(t_baboons, delta=1)
print(cl)
node_size = {v: 15 * (x / max(cl.values())) for v, x in cl.items()}
pp.plot(t_baboons, node_size=node_size, node_color=t_baboons.nodes);

We can also calculate the temporal betweenness centrality, which is based on the number of shortest time-respecting paths between pairs of nodes that pass through a given node. Again, this centrality score is sensitive to the time scale parameter $\delta$.

In [ ]:
bw = pp.algorithms.centrality.temporal_betweenness_centrality(t_baboons, delta=24*60)
print(bw)
node_size = {v: 15 * (x / max(bw.values())) for v, x in bw.items()}
pp.plot(t_baboons, node_size=node_size, node_color=t_baboons.nodes);

100%|██████████| 20/20 [00:00<00:00, 67.40it/s]


defaultdict(<function temporal_betweenness_centrality.<locals>.<lambda> at 0x7fe894f844c0>, {np.int64(0): 25.49506219995845, np.int64(9): 16.45944229302354, np.int64(10): 7.0646313362884605, np.int64(8): 10.16539885356489, np.int64(15): 18.70808072720025, np.int64(7): 2.5893301614085957, np.int64(5): 2.9692172804080914, np.int64(19): 8.975171000727638, np.int64(16): 0.7515158959863081, np.int64(18): 25.805683205683206, np.int64(17): 4.9153382565147306, np.int64(4): 17.320833765556717, np.int64(6): 2.1058097733020373, np.int64(12): 3.688175274115717, np.int64(1): 32.60701972686802, np.int64(2): 3.8280206113998165, np.int64(14): 2.9724179092311167, np.int64(3): 5.4875089420452685, np.int64(13): 6.091342786717137, np.int64(11): 1.7763568394002505e-15})


In [ ]:
bw = pp.algorithms.centrality.temporal_betweenness_centrality(t_baboons, delta=1)
print(bw)
node_size = {v: 15 * (x / max(bw.values())) for v, x in bw.items()}
pp.plot(t_baboons, node_size=node_size, node_color=t_baboons.nodes);

100%|██████████| 20/20 [00:00<00:00, 1046.87it/s]

defaultdict(<function temporal_betweenness_centrality.<locals>.<lambda> at 0x7fe894f84550>, {np.int64(0): 9.71445146547012e-15, np.int64(3): 1.999999999999984, np.int64(13): 0.99999999999998, np.int64(1): 1.0000000000000135, np.int64(12): -1.0047518372857667e-14, np.int64(18): 1.500000000000005, np.int64(7): 1.5543122344752192e-15, np.int64(8): -1.199040866595169e-14, np.int64(4): -1.4474532683550478e-14, np.int64(17): 0.4999999999999865, np.int64(2): 1.9999999999999858, np.int64(14): 1.0000000000000062, np.int64(5): 0.999999999999996, np.int64(16): 2.609024107869118e-15, np.int64(15): 0.9999999999999972, np.int64(6): -2.220446049250313e-16, np.int64(9): 1.0519363158323358e-14, np.int64(10): 3.3306690738754696e-15, np.int64(11): 0.0, np.int64(19): -1.1018963519404679e-14})
